Graph Classification

In [ ]:
import networkx as nx
import itertools
from collections import defaultdict

def read_graph_from_file(filepath):
    """Read a graph from a file with optional edge weights."""
    G = nx.Graph()
    with open(filepath, 'r') as file:
        for line in file:
            line = line.strip()
            if line.startswith('#') or not line:  
                continue
            parts = line.strip().split()
            if len(parts) >= 2:
                u, v = parts[0], parts[1]
                weight = float(parts[2]) if len(parts) == 3 else 1.0
                G.add_edge(u, v, weight=weight)
    return G

def find_cliques_dfs(G):
    """Find cliques of size >= 3 using DFS-based approach."""
    cliques = [clq for clq in nx.find_cliques(G) if len(clq) >= 3]
    return cliques

def find_chains_bfs(G):
    """Find chains of length >= 3 using BFS-based approach."""
    chains = []
    visited = set()
    
    for node in G.nodes():
        if node in visited or G.degree(node) > 2:
            continue
            
        # Start BFS from potential chain endpoints (degree 1 nodes)
        if G.degree(node) == 1:
            chain = [node]
            queue = [node]
            chain_visited = {node}
            
            while queue:
                current = queue.pop(0)
                neighbors = [n for n in G.neighbors(current) if n not in chain_visited]
                
                if len(neighbors) == 1 and G.degree(neighbors[0]) <= 2:
                    next_node = neighbors[0]
                    chain.append(next_node)
                    chain_visited.add(next_node)
                    queue.append(next_node)
                elif not neighbors and len(chain) >= 3:
                    chains.append(chain)
                    visited.update(chain)
                    break
                else:
                    break
    
    return chains

def find_stars_custom(G):
    """Find star configurations (1 central node connected to >= 3 nodes) using custom search."""
    stars = []
    
    for node in G.nodes():
        neighbors = list(G.neighbors(node))
        if len(neighbors) >= 3:
            # Check if the neighbors don't connect to each other (star property)
            if all(not G.has_edge(n1, n2) for n1, n2 in itertools.combinations(neighbors, 2)):
                stars.append([node] + neighbors)
    
    return stars

def find_cycles(G):
    """Find cycles of length >= 3 in an undirected graph using NetworkX's cycle_basis."""
    cycles = []
    # Use NetworkX's cycle_basis function which finds all elementary cycles
    cycle_basis = nx.cycle_basis(G)
    
    # Filter for cycles with length >= 3
    for cycle in cycle_basis:
        if len(cycle) >= 3:
            cycles.append(cycle)
    
    return cycles

def prioritize_subgraphs(subgraphs, G):
    """Prioritize subgraphs based on size, weight, and density."""
    scored = []
    for s_graph in subgraphs:
        size = len(s_graph)
        # Calculate total weight of the subgraph
        weight = sum(G[u][v].get('weight', 1) for u, v in itertools.combinations(s_graph, 2) 
                     if G.has_edge(u, v))
        
        # Calculate density (ratio of existing edges to possible edges)
        possible_edges = size * (size - 1) / 2
        sub_g = G.subgraph(s_graph)
        actual_edges = sub_g.number_of_edges()
        density = actual_edges / possible_edges if possible_edges > 0 else 0
        
        scored.append((size, weight, density, s_graph))
    
    # Sort by size (descending), then weight (descending), then density (descending)
    return sorted(scored, key=lambda x: (-x[0], -x[1], -x[2]))

def classify_subgraphs(G):
    """Classify subgraphs into cliques, chains, stars, and cycles."""
    result = defaultdict(list)
    
    # Use different search strategies for each type
    result['Cliques'] = prioritize_subgraphs(find_cliques_dfs(G), G)
    result['Chains'] = prioritize_subgraphs(find_chains_bfs(G), G)
    result['Stars'] = prioritize_subgraphs(find_stars_custom(G), G)
    result['Cycles'] = prioritize_subgraphs(find_cycles(G), G)
    
    return result


"""Main"""
path = input("Enter path to graph file: ")
graph = read_graph_from_file(path)
result = classify_subgraphs(graph)
for category, subgraphs in result.items():
        print(f"\n--- {category} ---")
        for i, (size, weight, density, s_graph) in enumerate(subgraphs, 1):
            print(f"{i}. Nodes: {s_graph}")
            print(f"   Size: {size}, Total Weight: {weight:.2f}, Density: {density:.2f}")